In [2]:
!pip install codecarbon
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Code generated using OpenAI O1 model
# The goal of this file is to see the
# emission tracker at work.

from codecarbon import EmissionsTracker
from datasets import load_dataset
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
    TrainingArguments,
    Trainer
)

# -----------------------------
# 1) Load a small subset of IMDb
# -----------------------------
dataset = load_dataset("imdb", split="train[:1000]")
dataset = dataset.train_test_split(test_size=0.2)

# For convenience, rename splits
train_dataset = dataset["train"]
eval_dataset  = dataset["test"]

# --------------------------------------------
# 2) Initialize the tokenizer and the model
# --------------------------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

# Tokenize dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset  = eval_dataset.map(preprocess_function,  batched=True)

# Keep only relevant columns
train_dataset = train_dataset.rename_column("label", "labels").remove_columns(["text"])
eval_dataset  = eval_dataset.rename_column("label", "labels").remove_columns(["text"])

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# --------------------------------------------
# 3) Set up training arguments
# --------------------------------------------
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="steps",
    eval_steps=10,
    logging_steps=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,   # For demonstration; increase for real use cases
    save_steps=50,
    disable_tqdm=False,
    load_best_model_at_end=False
)

# --------------------------------------------
# 4) Create the Trainer
# --------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# --------------------------------------------
# 5) Track emissions with CodeCarbon
# --------------------------------------------
tracker = EmissionsTracker(project_name="distilbert-imdb-demo")
tracker.start()

trainer.train()

tracker.stop()


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[codecarbon INFO @ 12:45:31] [setup] RAM Tracking...
[codecarbon INFO @ 12:45:31] [setup] CPU Tracking...
[codecarbon WARNING @ 12:45:31] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 12:45:33] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please co

Step,Training Loss,Validation Loss
10,0.184100,0.014939
20,0.008100,0.002620
30,0.002100,0.001046
40,0.000900,0.000629
50,0.000700,0.000450
60,0.000500,0.000352
70,0.000400,0.000290
80,0.000300,0.000248
90,0.000300,0.000217
100,0.000200,0.000194


[codecarbon INFO @ 12:45:49] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:45:49] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:45:49] Energy consumed for all GPUs : 0.000289 kWh. Total GPU Power : 69.42917685062864 W
[codecarbon INFO @ 12:45:49] 0.000486 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:46:04] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:46:04] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:46:04] Energy consumed for all GPUs : 0.000554 kWh. Total GPU Power : 63.46397985657393 W
[codecarbon INFO @ 12:46:04] 0.000948 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:46:19] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 12:46:19] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codeca